## Custom Model

### Tensorflow (-Slim)
First, define your model (its endpoints) and its preprocessing function.

In [1]:
from model_tools.activations.tensorflow import load_resize_image
import tensorflow as tf
slim = tf.contrib.slim
tf.reset_default_graph()

image_size = 224
placeholder = tf.placeholder(dtype=tf.string, shape=[64])
preprocess = lambda image_path: load_resize_image(image_path, image_size)
preprocess = tf.map_fn(preprocess, placeholder, dtype=tf.float32)

with tf.variable_scope('my_model', values=[preprocess]) as sc:
    end_points_collection = sc.original_name_scope + '_end_points'
    # Collect outputs for conv2d, fully_connected and max_pool2d.
    with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                        outputs_collections=[end_points_collection]):
        net = slim.conv2d(preprocess, 64, [11, 11], 4, padding='VALID', scope='conv1')
        net = slim.max_pool2d(net, [5, 5], 5, scope='pool1')
        net = slim.max_pool2d(net, [3, 3], 2, scope='pool2')
        net = slim.flatten(net, scope='flatten')
        net = slim.fully_connected(net, 1000, scope='logits')
        endpoints = slim.utils.convert_collection_to_dict(end_points_collection)

session = tf.Session()
session.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Then, using the TensorflowSlimWrapper, convert your model into an activations model.

In [2]:
from model_tools.activations.tensorflow import TensorflowSlimWrapper

activations_model = TensorflowSlimWrapper(identifier='tf-custom', labels_offset=0,
                                          endpoints=endpoints, inputs=placeholder, session=session)

Model candidates in Brain-Score have to follow the [model_interface](https://github.com/brain-score/brain-score/blob/master/brainscore/model_interface.py).
One of the major steps for most ML models to map onto the brain model interface is deciding what layers map onto cortical regions.
If you know which layer corresponds to which region beforehand, you can just use a `LayerModel` and `commit` a layer to a region.

In [3]:
from candidate_models import score_model
from model_tools.brain_transformation import LayerMappedModel

model = LayerMappedModel("tf-custom-pool2", activations_model=activations_model)
model.commit("IT", "my_model/pool2")

score = score_model(model_identifier=model.identifier, model=model,
                    benchmark_identifier='dicarlo.Majaj2015.IT-regressing')
print(score)

cross-validation: 100%|██████████| 10/10 [00:39<00:00,  3.98s/it]

<xarray.Score (aggregation: 2)>
array([ 0.494012, 10.52275 ])
Coordinates:
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:      <xarray.Score (aggregation: 2)>\narray([0.405688, 0.005272])\nC...
    ceiling:  <xarray.Score (aggregation: 2)>\narray([8.212101e-01, 5.010154e...


You can also empirically choose the best layer for a particular region. You might want to use PCA for large layers.

In [8]:
from model_tools.brain_transformation import LayerSelection
from model_tools.activations.pca import LayerPCA
from brainscore.assemblies.private import load_assembly

assembly = load_assembly('dicarlo.Majaj2015.lowvar.IT', average_repetition=False)
layer_selection = LayerSelection(model_identifier=model.identifier, activations_model=activations_model, 
                                 layers=['my_model/conv1', 'my_model/pool1', 'my_model/pool2'])
LayerPCA.hook(activations_model, n_components=1000)
best_layer = layer_selection(assembly)
print(f'Using layer {best_layer} for IT')

model = LayerMappedModel(model.identifier, activations_model=activations_model)
model.commit('IT', best_layer)

score = score_model(model_identifier=model.identifier, model=model,
                    benchmark_identifier='dicarlo.Majaj2015.IT-regressing')
print(score)

Using layer my_model/pool2 for IT
<xarray.Score (aggregation: 2)>
array([ 0.494012, 10.52275 ])
Coordinates:
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:      <xarray.Score (aggregation: 2)>\narray([0.405688, 0.005272])\nC...
    ceiling:  <xarray.Score (aggregation: 2)>\narray([8.212101e-01, 5.010154e...


In addition, you can resize stimuli according to visual degree angle like so:

In [9]:
from model_tools.brain_transformation import PixelsToDegrees

PixelsToDegrees.hook(activations_model, target_pixels=224)

All of these steps are sample implementations and as long as you implement the model_interface,
it does not matter how you get there.

## Pre-defined models

Scoring a model on neural data can be done in a single line using the `score_model` method and the `brain_translated_pool`.
Pre-defined layers of a model will be used to retrieve the activations.
Just like with the model implementations, the result of this method call will be cached 
so that it only needs to be computed once.


In [10]:
from candidate_models import score_model
from candidate_models.model_commitments import brain_translated_pool

identifier = 'alexnet-pca_1000-degrees'
model = brain_translated_pool[identifier]
score = score_model(model_identifier=identifier, model=model, benchmark_identifier='dicarlo.Majaj2015.IT-regressing')
print(score)


<xarray.Score (aggregation: 2)>
array([0.69516 , 4.740383])
Coordinates:
  * aggregation  (aggregation) <U6 'center' 'error'
Attributes:
    raw:      <xarray.Score (aggregation: 2)>\narray([0.570873, 0.002375])\nC...
    ceiling:  <xarray.Score (aggregation: 2)>\narray([8.212101e-01, 5.010154e...


## Score aggregate and raw values

A score typically comes with an estimate of the center (e.g. mean) and error (e.g. standard error of the mean).
These values are aggregations over splits and often neuroids.

We can also retrieve the raw scores from the same object, using `.raw` or `.attrs['raw']`.
The raw scores are for instance individual neuroids and splits.
In this case, we access raw twice: 1) to remove the ceiling, and 2) to remove the aggregation

In [13]:
raw = score.raw.raw
print(raw)

<xarray.DataAssembly (split: 10, neuroid: 168)>
array([[0.332988, 0.534487, 0.523227, ..., 0.662326, 0.708434, 0.689082],
       [0.355074, 0.510973, 0.445392, ..., 0.741481, 0.663006, 0.719654],
       [0.351747, 0.588668, 0.449489, ..., 0.716738, 0.764242, 0.744131],
       ...,
       [0.301904, 0.514429, 0.399632, ..., 0.716352, 0.695416, 0.70639 ],
       [0.245484, 0.48177 , 0.520564, ..., 0.721901, 0.715378, 0.725057],
       [0.315608, 0.537013, 0.54319 , ..., 0.751   , 0.718046, 0.715887]])
Coordinates:
  * split       (split) int64 0 1 2 3 4 5 6 7 8 9
  * neuroid     (neuroid) MultiIndex
  - neuroid_id  (neuroid) object 'Chabo_L_A_2_4' 'Chabo_L_A_3_3' ...
  - arr         (neuroid) object 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' ...
  - col         (neuroid) int64 4 3 5 0 1 2 3 4 5 6 2 3 5 0 1 2 3 4 5 6 1 2 ...
  - hemisphere  (neuroid) object 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' ...
  - subregion   (neuroid) object 'cIT' 'cIT' 'aIT' 'cIT' 'cIT' 'cIT' 'cIT' ...
  - a

The raw values are just another xarray object and as such, we can easily filter.

In [14]:
print(raw.sel(subregion='cIT'))

<xarray.DataAssembly (split: 10, neuroid: 75)>
array([[0.332988, 0.534487, 0.326301, ..., 0.416322, 0.752089, 0.568259],
       [0.355074, 0.510973, 0.334426, ..., 0.418433, 0.669294, 0.552998],
       [0.351747, 0.588668, 0.2921  , ..., 0.329157, 0.722749, 0.548502],
       ...,
       [0.301904, 0.514429, 0.232041, ..., 0.338111, 0.70119 , 0.599551],
       [0.245484, 0.48177 , 0.291308, ..., 0.356857, 0.704378, 0.477234],
       [0.315608, 0.537013, 0.364099, ..., 0.335651, 0.728882, 0.526765]])
Coordinates:
  * split       (split) int64 0 1 2 3 4 5 6 7 8 9
  * neuroid     (neuroid) MultiIndex
  - neuroid_id  (neuroid) object 'Chabo_L_A_2_4' 'Chabo_L_A_3_3' ...
  - arr         (neuroid) object 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' ...
  - col         (neuroid) int64 4 3 0 1 2 3 4 2 3 0 1 2 3 4 1 2 3 4 2 3 4 3 ...
  - hemisphere  (neuroid) object 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' ...
  - animal      (neuroid) object 'Chabo' 'Chabo' 'Chabo' 'Chabo' 'Chabo' ...
  - y   